<a href="https://colab.research.google.com/github/BeatrixBlaine/DS-C1/blob/main/pool_based_similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#demonstration generator

import random

def generate_demo():
    ops = ["+", "-", "×"]
    a, b = random.randint(1, 20), random.randint(1, 20)
    op = random.choice(ops)

    if op == "+":
        ans = a + b
    elif op == "-":
        ans = a - b
    else:
        ans = a * b

    return {
        "question": f"What is {a} {op} {b}?",
        "answer": str(ans),
        "op": op
    }


In [2]:
#build a pool

def build_pool(pool_size=50):
    return [generate_demo() for _ in range(pool_size)]


In [3]:
#random selection

def pool_based_selection(pool, k=4):
    return random.sample(pool, k)


In [4]:
def extract_op(question):
    for op in ["+", "-", "×"]:
        if op in question:
            return op
    return None


In [5]:
#similarity selection

def similarity_based_selection(pool, query, k=4):
    query_op = extract_op(query)

    scored = []
    for demo in pool:
        score = 1 if demo["op"] == query_op else 0
        scored.append((score, demo))

    scored.sort(reverse=True, key=lambda x: x[0])

    return [demo for _, demo in scored[:k]]


In [6]:
#prompt builder

def build_prompt(demos, query):
    prompt = "You are a grade school math solver.\n\n"
    for d in demos:
        prompt += f"Q: {d['question']}\nA: {d['answer']}\n\n"
    prompt += f"Q: {query}\nA:"
    return prompt


In [7]:
# Build synthetic pool
pool = build_pool(pool_size=30)

query = "What is 8 × 7?"

# Pool-based
pool_shots = pool_based_selection(pool, k=4)
pool_prompt = build_prompt(pool_shots, query)

# Similarity-based
sim_shots = similarity_based_selection(pool, query, k=4)
sim_prompt = build_prompt(sim_shots, query)

print("POOL-BASED PROMPT:\n", pool_prompt)
print("\nSIMILARITY-BASED PROMPT:\n", sim_prompt)


POOL-BASED PROMPT:
 You are a grade school math solver.

Q: What is 5 × 6?
A: 30

Q: What is 5 + 11?
A: 16

Q: What is 20 × 1?
A: 20

Q: What is 19 - 14?
A: 5

Q: What is 8 × 7?
A:

SIMILARITY-BASED PROMPT:
 You are a grade school math solver.

Q: What is 12 × 17?
A: 204

Q: What is 20 × 1?
A: 20

Q: What is 10 × 2?
A: 20

Q: What is 8 × 16?
A: 128

Q: What is 8 × 7?
A:
